In [1]:
include("data/10_ulysses_3.tsp")
n,L,B,K,W_v,w_v,W,coordinates

(10, 2, 87, 3, [1.47239, 0.82384, 0.147042, 1.11452, 1.35826, 1.95929, 1.98969, 1.54618, 2.14362, 1.6074], [4, 14, 3, 2, 20, 5, 14, 10, 10, 18], 9, [38.24 20.42; 39.57 26.15; … ; 41.23 9.1; 41.17 13.05])

In [2]:
using JuMP
using CPLEX

In [3]:
function distance(n::Int64,coordinates::Matrix{Float64})
    l=Matrix{Float64}(zeros(n,n))
    for i in 1:n
        for j in 1:n
            l[i,j]=sqrt((coordinates[i,1]-coordinates[j,1])^2+(coordinates[i,2]-coordinates[j,2])^2)
        end 
    end 
    return l
end

distance (generic function with 1 method)

In [4]:
l=distance(n,coordinates)

10×10 Matrix{Float64}:
  0.0        5.88233   5.42148   3.34819  …   0.720278  11.7082    7.93107
  5.88233    0.0       1.2919    4.48743      6.06684   17.1306   13.1973
  5.42148    1.2919    0.0       4.83011      5.74943   16.2338   12.2852
  3.34819    4.48743   4.83011   0.0          2.96142   14.8749   11.2033
 10.9669    16.7559   16.3883   12.8835      10.6926     7.88265   8.08926
  8.25804   14.104    13.4684   11.007    …   8.2501     4.7976    3.71102
  7.31222   13.0906   12.3961   10.2404       7.38505    4.89655   2.75065
  0.720278   6.06684   5.74943   2.96142      0.0       11.9315    8.24224
 11.7082    17.1306   16.2338   14.8749      11.9315     0.0       3.95046
  7.93107   13.1973   12.2852   11.2033       8.24224    3.95046   0.0

In [5]:
function resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
    
    l=distance(n,coordinates)
    m=Model(optimizer_with_attributes(CPLEX.Optimizer, "CPX_PARAM_MIPDISPLAY" =>2,"CPX_PARAM_TILIM" => 900))
    @variable(m,x[i=1:n,j=1:n],Bin)
    @variable(m,y[i=1:n,k=1:K],Bin)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]+y[i,k]-y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]-y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],-x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[k=1:K],sum(w_v[i]*y[i,k] for i=1:n)<=B)
    @constraint(m,[i=1:n],sum(y[i,k] for k=1:K)==1)
    @objective(m,Min,sum(l[i,j]*x[i,j] for i=1:n,j=1:n))
    return m 
end

resolution_statique (generic function with 1 method)

In [6]:
function resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
    l=distance(n,coordinates)
    m=Model(optimizer_with_attributes(CPLEX.Optimizer, "CPX_PARAM_MIPDISPLAY" =>2,"CPX_PARAM_TILIM" => 900))
    @variable(m,alpha>=0)
    @variable(m,beta[i=1:n,j=1:n]>=0)
    @variable(m,x[i=1:n,j=1:n],Bin)
    @variable(m,y[i=1:n,k=1:K],Bin)
    @variable(m,gamma[k=1:K]>=0)
    @variable(m,phi[i=1:n,k=1:K]>=0)



    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]+y[i,k]-y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]-y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],-x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n],alpha+beta[i,j]>=(lh[i]+lh[j])*x[i,j])
    @constraint(m,[k=1:K],sum(w_v[i]*y[i,k]+W_v[i]*phi[i,k] for i=1:n)+W*gamma[k]<=B)
    @constraint(m,[i=1:n,k=1:K],gamma[k]+phi[i,k]>=w_v[i]*y[i,k])
    @constraint(m,[i=1:n],sum(y[i,k] for k=1:K)==1)

    @objective(m,Min,sum(l[i,j]*x[i,j]+3*beta[i,j] for i=1:n,j=1:n)+L*alpha)
    optimize!(m)
    
    return sum(l[i,j]*value.(x)[i,j] for i =1:n, j=1:n),solve_time(m)
end

resolution_dualite (generic function with 1 method)

In [8]:
include("data/10_ulysses_3.tsp")
ms=resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
optimize!(ms)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_TimeLimit                               900
Tried aggregator 1 time.
MIP Presolve eliminated 90 rows and 10 columns.
MIP Presolve modified 24 coefficients.
Reduced MIP has 823 rows, 120 columns, and 2490 nonzeros.
Reduced MIP has 120 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.14 ticks)
Found incumbent of value 220.697504 after 0.00 sec. (3.07 ticks)
Probing time = 0.00 sec. (0.20 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 823 rows, 120 columns, and 2490 nonzeros.
Reduced MIP has 120 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (4.02 ticks)
Probing time = 0.00 sec. (0.20 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (1.33 ticks)

        Nodes                            

In [9]:
resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)


Zero-half cuts applied:  29

Root node processing (before b&c):
  Real time             =    0.09 sec. (69.01 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.09 sec. (69.01 ticks)
Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_TimeLimit                               900
Tried aggregator 1 time.
MIP Presolve eliminated 190 rows and 111 columns.
MIP Presolve modified 70 coefficients.
Reduced MIP has 853 rows, 153 columns, and 2613 nonzeros.
Reduced MIP has 120 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.96 ticks)
Found incumbent of value 251.376334 after 0.00 sec. (4.36 ticks)
Probing time = 0.00 sec. (0.28 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 853 rows, 153 columns, and 2613 nonzeros.
Reduced MIP has 120 binaries, 0 general

(189.99055259178834, 0.13899993896484375)

In [14]:
println(sum(l[i,j]*value.(x)[i,j] for i =1:n, j=1:n))


Implied bound cuts applied:  6
Flow cuts applied:  3
Mixed integer rounding cuts applied:  14
Zero-half cuts applied:  41

Root node processing (before b&c):
  Real time             =    5.13 sec. (95.76 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    5.13 sec. (95.76 ticks)


UndefVarError: UndefVarError: x not defined

In [ ]:
data=["data/10_ulysses_3.tsp","data/10_ulysses_6.tsp","data/10_ulysses_9.tsp","data/14_burma_3.tsp","data/14_burma_6.tsp","data/14_burma_9.tsp","data/22_ulysses_3.tsp","data/22_ulysses_6.tsp","data/22_ulysses_9.tsp","data/26_eil_3.tsp"]

In [ ]:
for path in data 
    open("results_statique.txt","a") do file 
        include(path)
        ms=resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
        optimize!(ms)
        println(file,string(n)*"&"*string(K)*"&"*string(objective_value(ms))*"&"*string(solve_time(ms)))
    end
end

In [ ]:
for path in data 
    open("results_dualité.txt","a") do file 
        include(path)
        ms=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
        println(file,string(n)*"&"*string(K)*"&"*string(ms[1])*"&"*string(ms[2]))
    end
end

In [7]:
path="data/22_ulysses_9.tsp"
open("results_dualité.txt","a") do file 
    include(path)
    ms=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
    optimize!(ms)
    println(file,string(n)*"&"*string(K)*"&"*string(objective_value(ms))*"&"*string(solve_time(ms)))
end

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_TimeLimit                               900
Tried aggregator 1 time.
MIP Presolve eliminated 1078 rows and 507 columns.
MIP Presolve modified 304 coefficients.
Reduced MIP has 12703 rows, 867 columns, and 38619 nonzeros.
Reduced MIP has 660 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (24.19 ticks)
Found incumbent of value 315.933889 after 0.09 sec. (57.41 ticks)
Probing time = 0.00 sec. (2.31 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 12703 rows, 867 columns, and 38619 nonzeros.
Reduced MIP has 660 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.22 sec. (187.86 ticks)
Probing time = 0.02 sec. (2.14 ticks)
Clique table members: 22.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.06 sec. (36.67 ticks)

        Nodes             

In [8]:
path="data/26_eil_3.tsp"
open("results_dualité.txt","a") do file 
    include(path)
    ms=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
    optimize!(ms)
    println(file,string(n)*"&"*string(K)*"&"*string(objective_value(ms))*"&"*string(solve_time(ms)))
end


Cover cuts applied:  4
Implied bound cuts applied:  88
Flow cuts applied:  13
Mixed integer rounding cuts applied:  39
Zero-half cuts applied:  76
Lift and project cuts applied:  1
Gomory fractional cuts applied:  9

Root node processing (before b&c):
  Real time             =    6.31 sec. (3077.04 ticks)
Parallel b&c, 8 threads:
  Real time             =  899.88 sec. (195711.98 ticks)
  Sync time (average)   =  166.45 sec.
  Wait time (average)   =    0.02 sec.
                          ------------
Total (root+branch&cut) =  906.19 sec. (198789.02 ticks)
Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_TimeLimit                               900
Tried aggregator 1 time.
MIP Presolve eliminated 234 rows and 26 columns.
MIP Presolve modified 182 coefficients.
Reduced MIP has 6633 rows, 1486 columns, and 20023 nonzeros.
Reduced MIP has 728 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.26 sec. (9.73 ticks)
Found incumbent of value 103125.298749 after 1.